# Scenario models

For a delta-change in a feature (or group of features), what is the likely change in displacement?

### Assumptions:
- changes and impacts occur in year=0, year=1
- changes across themes are additive
- User-specified change are a percentage change. 
- 2 geographies - 'AFG, 'MMR'
  
### Notes
- groupings.json - need to be verified with DRC.


In [1]:
import pandas as pd
from time import time
import os
import json
import pickle
import numpy as np 
from time import time
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

# Data transforms

<TBC>

In [2]:
start_time = time()
with open("../configuration.json", 'rt') as infile:
    config = json.load(infile)


sources = [os.path.join("..", config['paths']['output'], 
                        d['name'], 
                        'data.csv') for d in config['sources'] if d['name']]


# Generate a data frame with all indicators
df = pd.concat((pd.read_csv(f) for f in sources), sort=False, ignore_index=True)

# Summary stats
print("Sources            : {}".format(len(sources)))
print("Shape              : {} (rows) {} (columns)".format(*df.shape))
print("Geographies        : {}".format(len(df['Country Name'].unique())))
print("Indicators         : {}".format(len(df['Indicator Code'].unique())))
print("Temporal coverage  : {} -> {}".format(df.year.min(), df.year.max()))
print("Null values        : {}".format(sum(df['value'].isnull())))

print("\nLoaded data in {:3.2f} sec.".format(time() - start_time))


# Now arrange data in wide form
data = pd.pivot_table(df, index=['Country Code', 'year'],
                      columns='Indicator Code', values='value')

# Consider country/year as features (and not an index)
data.reset_index(inplace=True)

print("Long form of size  : {} (rows) {} (columns)".format(*data.shape))

Sources            : 19
Shape              : 1016946 (rows) 6 (columns)
Geographies        : 562
Indicators         : 151
Temporal coverage  : 1789 -> 2019
Null values        : 1421

Loaded data in 1.27 sec.
Long form of size  : 35858 (rows) 153 (columns)


In [18]:
c1 = data['Country Code'] == 'MMR'

v = 'ADESA' #['EMDAT.NAT.OCCURRENCE']# ,  'ADESA'] # ['ER.H2O.INTR.PC'] #
#plt.scatter(data.loc[c1, 'DRC.TOT.DISP'], data.loc[c1, v])

Y = data.loc[c1, 'DRC.TOT.DISP']
X = data.loc[c1, v]

# handle missing labels
idx = ~pd.isnull(Y.values)
Y = Y[idx]
X = X[idx]

# Missing data
X.fillna(X.mean(), inplace=True)
Y.fillna(Y.mean(), inplace=True)
X1 = sm.add_constant(X)

m = sm.OLS(Y, X1)
res = m.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:           DRC.TOT.DISP   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.2769
Date:                Mon, 24 Feb 2020   Prob (F-statistic):              0.601
Time:                        13:31:00   Log-Likelihood:                -771.97
No. Observations:                  53   AIC:                             1548.
Df Residuals:                      51   BIC:                             1552.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -8.735e+04   1.14e+06     -0.077      0.9

In [4]:
# Only look at features used in scenarios
groupings = json.load(open("../groupings.json", 'rt'))
featureset = [i['code'] for c in groupings['clusters'] for i in c['indicators']]

# Dimensions of interest
CLUSTERS = groupings['clusters']
COUNTRIES = ['AFG', 'MMR']
LAGS = [0, 1]
LABELS = ['-10%', '-5%', '0%', '+5%', '+10%']


def lag_variables(data, var, lag):
    """
    Append lagged variables to frame.
    
    data - pandas data frame
    var - list of variable names to lag
    lag - integer
    
    """
    idx_cols = ['year', 'Country Code']
    fv = var + idx_cols
    
    tmp = data[fv].copy(deep=True)
    
    col_name = [v + ".T" + "{0:+}".format(lag) for v in var]
    
    tmp.rename(columns={k: v for (k, v) in zip(var, col_name)},
               inplace=True)
    tmp.year -= lag
    data = pd.merge(data, tmp, on=idx_cols, how='left')
    
    return data, col_name


def model_case(data, lag, countries):
    """ Generate a data frame to estimate elasticities """
    
    # Features
    idx = ['Country Code', 'year']
    TARGETS = ['DRC.TOT.DISP']

    # spatial filter
    if countries.lower() != 'global':
        if ~isinstance(countries, list):
            countries = [countries]
        c1 = data['Country Code'].isin(countries)
    else:
        # use all countries
        c1 = [True] * len(data)
    
    if lag != 0:
        df, TARGETS = lag_variables(data, TARGETS, lag)
    else:
        df = data.copy(deep=True)
        
    # temporal filter
    c2 = data.year >= 1980

    df = df.loc[c1 & c2, idx + featureset + TARGETS]
    
    X = df.loc[:, featureset]
    Y = df.loc[:, TARGETS]
    
    # handle missing labels
    idx = ~pd.isnull(Y.values)
    Y = Y[idx]
    X = X[idx]
    
    # Missing data
    X.fillna(X.mean(), inplace=True)
    Y.fillna(Y.mean(), inplace=True)

    return X, Y

In [5]:

for c in COUNTRIES:
    for v in featureset:
        c1 = data['Country Code'] == c

        Y = data.loc[c1, 'DRC.TOT.DISP']
        X = data.loc[c1, v]

        # handle missing labels
        idx = ~pd.isnull(Y.values)
        Y = Y[idx]
        X = X[idx]

        # Missing data
        X.fillna(X.mean(), inplace=True)
        Y.fillna(Y.mean(), inplace=True)
        X1 = sm.add_constant(X)

        m = sm.OLS(Y, X1)
        res = m.fit()
        
        fig = plt.figure(figsize=(12,8))
        fig = sm.graphics.plot_regress_exog(res, v, fig=fig)
        plt.savefig("img/regplots/{}-{}.png".format(c, v))
        plt.close()



# Assemble and build

In [6]:
from itertools import product

models = {}
for lag, c in product(LAGS, COUNTRIES):
    
    X, Y = model_case(data, lag, c)
    
    X1 = sm.add_constant(X)

    # Pick a random validation sample
    Xv = X1.head(1)
    
    M = {}
    key = c, lag
    clf = sm.OLS(Y, X1)
    m = clf.fit()
    M['significance'] = (m.pvalues<0.05).to_dict()
    M['elasticity'] = m.params.to_dict()
    M['model'] = m
    M['Xv'] = X1.head(1)
    models[key] = M


In [9]:
theme_indicator_map = {c['theme']: c['indicators'] for c in CLUSTERS}

def get_significance(pvalues, indicators):
    
    c = [i['code'] for i in indicators]
    s = [pvalues[i] for i in c]

    if all(s):
        # Statistically significant for entire cluster
        return "*"
    elif any(s):
        # only some indicators within cluster are statistically significant
        return "x"
    else:
        # Not statistically significant
        return "ns"
    
def elasticity(M, scenario):
    """
    Compute elasticity of target variable relative to a 
    user-specified scenario.
    M - linear models with variable level groupings
    scenario - dictionary of scenario inputs
    
    returns change in displacement
    """

    e = M['elasticity']
    m = M['model']
    sig = M['significance']
    Xv = M['Xv']
    
    bl = m.predict(Xv).values[0]
    # Numeric equivalent
    user_scenario = {}

    for theme, change in scenario.items():

        # get the percentage change set
        ds = float(change.replace('%', '')) / 100.0
        indicatorset = theme_indicator_map[theme]

        # Is the change statistically significant
        significance = get_significance(sig, indicatorset)
        
        for i in indicatorset:
            if i['direction-improvement'] == 'lower':
                # flip the direction
                user_scenario[i['code']] = -1 * ds
            else:
                user_scenario[i['code']] =  ds

    # numerical change

    num_change = {k: v * np.abs(Xv[k].values[0]) for k, v in user_scenario.items()}
    b = Xv.to_dict(orient='records')[0]
    for k, v in num_change.items():
        if ~(np.abs(v - b[k]) <= 1e-5):
            verb = 'reduced' if v <=0 else 'increased'
                
            #print("{} {} from {:.1f} by {:.1f} (significant? {}) dt: {:.0f}, e: {:.0f}".format(k, verb, b[k], v, sig[k], e[k] * v, e[k]))
    
    ind_dt = [e[k] * v for k, v in num_change.items()]

    # finally report change relative to baseline
    dt = sum(ind_dt)/bl
    return dt, significance
    # verb = 'increase' if dt > 0 else 'decrease'
    #print("{} change results in {:.2%} {} in forced displacement ({}).".format(scenario, np.abs(dt), verb, significance))



In [11]:
result = []
for cl, l, country in product(CLUSTERS, LABELS, COUNTRIES):
    
    lag = 0
    scenario = {cl['theme']: l}
    key = (country, lag)
    M = models[key]
    
    dt, sig = elasticity(M, scenario)

    result.append({
        'theme': cl['theme'],
        'change-ind': l,
        'country': country,
        'lag': lag,
        'change-disp': dt,
        'significance': sig
    })

In [12]:
rdf = pd.DataFrame(result)
rdf['value'] = rdf.apply(lambda x: "{:+1.0f}% {}".format(100*x['change-disp'], x['significance']), axis=1)
rdf.head()

,theme,change-ind,country,lag,change-disp,significance,value
0,Economy,-10%,AFG,0,-0.054209,x,-5% x
1,Economy,-10%,MMR,0,3.693538,ns,+369% ns
2,Economy,-5%,AFG,0,-0.027104,x,-3% x
3,Economy,-5%,MMR,0,1.846769,ns,+185% ns
4,Economy,0%,AFG,0,0.000000,x,+0% x


In [13]:
#rdf.set_index(['country', 'theme', 'lag'])

tmp = rdf[rdf['lag']== 0]

tmp = pd.pivot_table(rdf, index=['country', 'theme', 'lag'], 
                     columns='change-ind', values='value',
                    aggfunc=lambda x: ' '.join(x)) 

tmp = tmp[LABELS]

In [14]:
tmp

change-ind                   -10%       -5%      0%       +5%       +10%
country theme      lag                                                  
AFG     Conflict   0       +12% x     +6% x   +0% x     -6% x     -12% x
        Economy    0        -5% x     -3% x   +0% x     +3% x      +5% x
        Governance 0      +52% ns   +26% ns  +0% ns   -26% ns    -52% ns
        Natural    0      -18% ns    -9% ns  +0% ns    +9% ns    +18% ns
        Population 0       -2% ns    -1% ns  +0% ns    +1% ns     +2% ns
MMR     Conflict   0     -164% ns   -82% ns  +0% ns   +82% ns   +164% ns
        Economy    0     +369% ns  +185% ns  +0% ns  -185% ns   -369% ns
        Governance 0    -1129% ns  -564% ns  +0% ns  +564% ns  +1129% ns
        Natural    0     -131% ns   -66% ns  +0% ns   +66% ns   +131% ns
        Population 0     -1475% x   -738% x   +0% x   +738% x   +1475% x

#### Summary

- Results for Myanmar are poor. Besides population most categories are not statistically significant. The magnitude and signs of changes in displacement suggest that the OLS method is unsuitable for this case. 
- For AFG, elasticities relative to the thematic clusters make intuitive sense. The lag effects also imply, a decay in conflict impact, and a delayed impact of economy. 
